<a href="https://colab.research.google.com/github/LeeEunjeong1/-sentiment-analysis/blob/main/%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [8]:
import io
import pandas as pd

#data1
filename = '/content/drive/My Drive/drive-download-20211229T031307Z-001/Reviews.csv'
df = pd.read_csv(filename)
df = df[df['Score'] != 3]
df['sentiment']=df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
review_df1 = df[['Text','sentiment']]

#data2
filename2 = '/content/drive/My Drive/drive-download-20211229T031307Z-001/Tweets.csv'
df = pd.read_csv(filename2)
review_df2 = df[['text','airline_sentiment']]
review_df2 = review_df2[review_df2['airline_sentiment'] != 'natural']
review_df2.columns = ["Text","sentiment"]

#data concatanation
review_df = pd.concat([review_df1,review_df2],ignore_index=True)
print(review_df)
sentiment_label = review_df.sentiment.factorize()
test_sets = review_df.Text.values

                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
540449  @AmericanAir thank you we got on a different f...  positive
540450  @AmericanAir leaving over 20 minutes Late Flig...  negative
540451  @AmericanAir Please bring American Airlines to...   neutral
540452  @AmericanAir you have my money, you change my ...  negative
540453  @AmericanAir we have 8 ppl so we need 2 know h...   neutral

[540454 rows x 2 columns]


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(test_sets)
print(tokenizer.word_index,len(tokenizer.word_index))

{'the': 1, 'i': 2, 'and': 3, 'a': 4, 'to': 5, 'it': 6, 'of': 7, 'is': 8, 'this': 9, 'br': 10, 'for': 11, 'in': 12, 'my': 13, 'that': 14, 'but': 15, 'with': 16, 'have': 17, 'you': 18, 'are': 19, 'not': 20, 'was': 21, 'they': 22, 'on': 23, 'as': 24, 'so': 25, 'like': 26, 'these': 27, 'them': 28, 'good': 29, 'be': 30, 'at': 31, 'or': 32, 'one': 33, 'great': 34, 'just': 35, 'very': 36, 'taste': 37, 'product': 38, 'if': 39, 'all': 40, 'coffee': 41, 'from': 42, 'flavor': 43, 'tea': 44, 'can': 45, 'we': 46, 'when': 47, "it's": 48, 'more': 49, 'has': 50, 'love': 51, 'me': 52, 'food': 53, 'will': 54, 'had': 55, 'would': 56, 'out': 57, 'than': 58, 'no': 59, 'get': 60, 'other': 61, 'amazon': 62, 'up': 63, 'some': 64, 'about': 65, 'only': 66, 'really': 67, 'too': 68, 'much': 69, 'your': 70, 'an': 71, "don't": 72, 'also': 73, 'time': 74, 'use': 75, 'because': 76, 'little': 77, 'best': 78, 'do': 79, 'been': 80, 'were': 81, 'what': 82, 'there': 83, 'tried': 84, 'buy': 85, 'which': 86, 'price': 87, 'e

In [10]:
encoded_docs = tokenizer.texts_to_sequences(test_sets)

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)
print(padded_sequence)

[[    0     0     0 ...   101    58   141]
 [    0     0     0 ...    38    24  5786]
 [    0     0     0 ...     5     1 13925]
 ...
 [    0     0     0 ...  1225  6798     5]
 [    0     0     0 ...    96    13  8113]
 [    0     0     0 ...     1   411  1024]]


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size = len(tokenizer.word_index)+1
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size,embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           4283040   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,299,691
Trainable params: 4,299,691
Non-trainable params: 0
______________________________________________

In [13]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=1, batch_size=512)

845/845 [==============================] - 879s 1s/step - loss: 0.2277 - accuracy: 0.9129 - val_loss: 0.3476 - val_accuracy: 0.8627


In [25]:
epochs=6
batch_size=64
history = model.fit(padded_sequence, sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+',batch_size'+str(batch_size)+').h5')

Epoch 1/6
  4/865 [..............................] - ETA: 24:49 - loss: 0.0746 - accuracy: 0.9760

KeyboardInterrupt: ignored

In [ ]:
epochs=2
batch_size=10
history = model.fit(padded_sequence, sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
model.save('trained_model(epoch='+str(epochs)+',batch_size'+str(batch_size)+'),h5')

KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


In [18]:
#epochs=7
#batch_size=128
#history = model.fit(padded_sequence, sentiment_label[0],validation_split=0.2, epochs=epochs, batch_size=batch_size)
#model.save('trained_model(epoch='+str(epochs)+',batch_size'+str(batch_size)+'),h5')

In [23]:
# 감정  분석하기
from keras.models import load_model
model = load_model('trained_model|(epoch=6,batch_size=64).h5')
print(model.summary())

OSError: ignored

In [1]:
import io
import pandas as pd

#data1
filename = '/content/drive/My Drive/drive-download-20211229T031307Z-001/Reviews.csv'
df = pd.read_csv(filename)
df = df[df['Score'] != 3]
df['sentiment']=df['Score'].apply(lambda rating : 'positive' if rating > 3 else 'negative')
review_df1 = df[['Text','sentiment']]
test_sets1=review_df1.Text.values

#data2
filename2 = '/content/drive/My Drive/drive-download-20211229T031307Z-001/Tweets.csv'
df = pd.read_csv(filename2)
review_df2 = df[['text','airline_sentiment']]
review_df2 = review_df2[review_df2['airline_sentiment'] != 'natural']
review_df2.columns = ["Text","sentiment"]
test_Sets2=review_df2.Text.values

review_df=pd.concat([review_df1,review_df2],ignore_index=True)
print(review_df)
test_sets=review_df.Text.values
sentiment_label = review_df.sentiment.factorize()
answers=review_df.sentiment.values

length=len(test_sets1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


                                                     Text sentiment
0       I have bought several of the Vitality canned d...  positive
1       Product arrived labeled as Jumbo Salted Peanut...  negative
2       This is a confection that has been around a fe...  positive
3       If you are looking for the secret ingredient i...  negative
4       Great taffy at a great price.  There was a wid...  positive
...                                                   ...       ...
540449  @AmericanAir thank you we got on a different f...  positive
540450  @AmericanAir leaving over 20 minutes Late Flig...  negative
540451  @AmericanAir Please bring American Airlines to...   neutral
540452  @AmericanAir you have my money, you change my ...  negative
540453  @AmericanAir we have 8 ppl so we need 2 know h...   neutral

[540454 rows x 2 columns]


In [2]:
#predict function
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(test_sets)

encoded_docs = tokenizer.texts_to_sequences(test_sets)

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

def predict_sentiment(text):
  tw = tokenizer.texts_to_sequences([text])
  tw = pad_sequences(tw,maxlen=200)
  prediction = int(model.predict(tw).round().item())
  return sentiment_label[1][prediction]

In [3]:
test_sentence1 = "I enjoyed my journey on this flight."
test_sentence2 = "This is the worst flight experience of my life!" 
test_sentence3 = "This is goods are bad"
test_sentence4 = "This goods are so so"

print(predict_sentiment(test_sentence1),predict_sentiment(test_sentence2),predict_sentiment(test_sentence3),predict_sentiment(test_sentence4))

NameError: ignored

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
stopwords.update(["br","href","would"])

positives = dict()
negatives=dict()
pos_cnt=0
neg_cnt=0
cnt=0

meta_text=""
pos_meta_text=""
neg_meta_text=""
for i in range(length):
  text= re.sub('^@[a-zA-Z0-9*','',test_sets1[i])
  words=text.split(' ')
  words=[word for word in words if word not in stopwords]
  text = ' '.join(words)
  text=''.join(list(filter(lambda x: x not in "!@#$%^&*+()-='",text)))
  text=str(text).lower()
  text = re.sub('\[.*?\]','',text)
  text = re.sub('https?://\S+|www\.\S+','',text)
  text = re.sub('<.*?>+','',text)
  text = re.sub(r'[^\w\s]','',text)
  text = re.sub('\n','',text)
  text = re.sub('\w*\d\w*','',text)
  result=predict_sentiment(text)
  words=text.split(' ')
  words= list(filter(None, words))
  words=[word for word in words if word not in stopwords]

  text= " ".join(words)
  meta_text+=text+" "

  if result=='positive':
    pos_cnt+=1
    for word in words:
      if word in positives:
        positives[word]+=1
      else:
        positives[word]=1
    pos_meta_text+=text+" "

  else:
    neg_cnt+=1
    for word in words:
      if word in negatives:
        negatives[word]+=1
      else:
        negatives[word]=1
    neg_meta_text+=text+" "

  if pos_cnt%500==0 or neg_cnt%500==0:
    print(pos_cnt, neg_cnt)
  if result==answers[i]:
    cnt+=1
  else:
    print(i+1, result, answers[i], "%d/%d"%(cnt,i+1))

wordcloud = WordCloud(stopwords=stopwords).generate(meta_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud_total.png', pad_inches=0.1)
plt.show()


In [ ]:
import operator

piositives=dict(positives)
negatives=dict(negatives)

sPositives= sorted(positives.items(), key=operator.itemgetter(1),reverse=True)
sNegatives= sorted(negatives.items(), key=operator.itemgetter(1),reverse=True)

sPositives=dict(sPositives)
sNegatives=dict(sNegatives)
pos_keys=list(sPositives.keys())
pos_values=list(sPositives.values())
neg_keys=list(sNegatives.keys())
neg_values=list(sNegatives.values())

top_pos_keys=pos_keys[:80]
top_pos_cnt=pos_values[:80]
top_neg_keys=neg_keys[:80]
top_neg_cnt=neg_values[:80]

cnt=0
i=0
while i<len(top_pos_keys):
  if top_pos_keys[i] in top_neg_keys:

    j=top_neg_keys.index(top_pos_keys[i])
    del top_pos_keys[i]
    del top_pos_cnt[i]
    del top_neg_keys[j]
    del top_neg_cnt[j]
  
  else:
    i+=1
print(top_pos_keys)

In [ ]:
wordcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_pos_keys)) 
plt.imshow(wordcloud, interpolation='bilinear') 
plt.axis("off") 
plt.savefig('wordcloud_pos.png', pad_inches=0.1) 
plt.show()

wordcloud = WordCloud(stopwords=stopwords).generate(' '.join(top_neg_keys)) 
plt.imshow(wordcloud, interpolation='bilinear') 
plt.axis("off") 
plt.savefig('wordcloud_neg.png', pad_inches=0.1) 
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

X=['positive', 'negative']
freg=[pos_cnt, neg_cnt]

plt.bar(X, freg, color=['limegreen', 'red']) 
plt.savefig('./count.png', dpi=200, bbox_inches='tight', pad_inches=0.1) 
plt.show()

top_pos_keys=top_pos_keys[:20] 
top_pos_cnt=top_pos_cnt [:20]

top_neg_keys=top_neg_keys[:20] 
top_neg_cnt=top_neg_cnt [:20]
#mosf fop freguenf 20 positive xords 
colors = sns.color_palette('YIG',20) 
plt.barh(top_pos_keys[::-1], top_pos_cnt [::-1], color=colors)
plt.savefig('./positive.png', dpi=200, bbox_inches='tight', pad_inches=0.1) 
plt.show
#most fop frequent 20 negative words 
colors = sns.color_palette('Reds',20) 
plt.barh(top_neg_keys[::-1], top_neg_cnt [::-1], color=colors) 
plt.savefig('./negative.png', dpi=200, bbox_inches='tight', pad_inches=0.1) 
plt.show()


In [ ]:
from docx import Document
from docx.shared import Inches 
from docx.shared import Pt

document = Document() 
document.add_heading('Sentiment Analysis Result', level = 0) 
p = document.add_paragraph(style='ListNumber') 
p.add_run().font.size=Pt(12) 
p.add_run('Classification').bold=True 
document.add_picture('count.png', width=Inches(5,5)) 
p = document.add_paragraph("The number of positive responses: "+str(pos_cnt)+ " ("+str(round(pos_cnt*100/(pos_cnt+neg_cnt), 2))+"%"
p = document.add_paragraph("The number of negative responses: "+str(neg_cnt)+ " ("+str(round(neg_cnt +100/(pos_cnt +neg_cnt),2))+"%"
document.add_picture( 'wordcloud_total.png', width=Inches(5)) 
#document.add_page_break()

p = document, add_paragraph(style='ListNumber' ) 
p.add_run('Positive').bold=True 
p = document.add_paragraph ("The number of positive response: "+str(pos_cnt), style='ListBullet2') 
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2') 
document.add_picture('positive.png', width=Inches(5.5))

table = document.add_table(rows = 21, cols = 2, style = "Table Grid") 
for i in range(21):
  table.cell(i,0).width = Inches(1.25)
  table.cell(i,1).width = Inches(1.25) 
table.rows[0].cells[0].text = "word" 
table.rows[0].cells[1].text = "count" 
for i in range(20):
  table.rows[i+1].cells[0].text = top_pos_keys[i]
  table.rows[i+1].cells[1].text = str(top_pos_cnt[i]) 
document.add_picture('wordcloud_pos.png', width=Inches(5)) 
document.add_page_break()

p = document.add_paragraph(style='ListNumber') 
p.add_run('Negative').bold=True


In [ ]:
p = document.add_paragraph("The number of negative response: "+str(neg_cnt), style='ListBullet2') 
p = document.add_paragraph("Top 20 frequent words: ", style='ListBullet2') 
document.add_picture('negative.png', width=Inches(5.5))

table = document.add_table(rows = 21, cols = 2, style = "Table Grid") 
for i in range(21):
  table.cell(i,0).width = Inches(1.25)
  table.cell(i,1).width = inches (1.25) 
table.rows[O].cells[0].text = "word" 
table.rows[0].cells[1].text = "count"

for i in range (20):
   # if len/neg_keys)<=: 
   # continue
  table.rows[i+1].cells[0].text = top_neg_keys[i]
  table.rows[i+1].cells[1].text = str(top_neg_cnt[i]) 
document.add_picture('wordcloud_neg.png', width=Inches(5))

document.save("result.docx")
